# Loading data into memory

Loading API is central to a lot of nilmtk operations and provides a great deal of flexibility. Let's look at ways in which we can load data from a NILMTK DataStore into memory. To see the full range of possible queries, we'll use the [iAWE data set](http://iawe.github.io) (whose HDF5 file can be downloaded [here](https://copy.com/C2sIt1UfDx1mfPlC)).

The `load` function returns a *generator* of DataFrames loaded from the DataStore based on the conditions specified. If no conditions are specified, then all data from all the columns is loaded.  (If you have not come across Python generators, it might be worth reading [this quick guide to Python generators](http://stackoverflow.com/a/1756156/732596).)

In [11]:
from nilmtk import DataSet

iawe = DataSet('/nilmtk/data/ukdale.h5') #('/nilmtk/data/iawe.h5')
elec = iawe.buildings[1].elec
elec

MeterGroup(meters=
  ElecMeter(instance=2, building=1, dataset='UK-DALE', appliances=[Appliance(type='boiler', instance=1)])
  ElecMeter(instance=3, building=1, dataset='UK-DALE', appliances=[Appliance(type='solar thermal pumping station', instance=1)])
  ElecMeter(instance=4, building=1, dataset='UK-DALE', appliances=[Appliance(type='laptop computer', instance=1), Appliance(type='laptop computer', instance=3)])
  ElecMeter(instance=5, building=1, dataset='UK-DALE', appliances=[Appliance(type='washer dryer', instance=1)])
  ElecMeter(instance=6, building=1, dataset='UK-DALE', appliances=[Appliance(type='dish washer', instance=1)])
  ElecMeter(instance=7, building=1, dataset='UK-DALE', appliances=[Appliance(type='television', instance=1)])
  ElecMeter(instance=8, building=1, dataset='UK-DALE', appliances=[Appliance(type='light', instance=1), Appliance(type='light', instance=2)])
  ElecMeter(instance=9, building=1, dataset='UK-DALE', appliances=[Appliance(type='HTPC', instance=1)])
  Ele

Let us see what measurements we have for the fridge:

In [14]:
fridge = elec['fan']
fridge.available_columns()

[('power', 'active')]

## Loading data

### Load all columns (default)

In [15]:
df = next(fridge.load())
df.head()

physical_quantity,power
type,active
2013-06-19 15:26:41+01:00,0
2013-06-19 15:26:45+01:00,6
2013-06-19 15:26:51+01:00,0
2013-06-19 15:26:58+01:00,0
2013-06-19 15:27:04+01:00,26


### Load a single column of power data

Use `fridge.power_series()` which returns a generator of 1-dimensional `pandas.Series` objects, each containing power data using the most 'sensible' AC type:

In [16]:
series = next(fridge.power_series())
series.head()

2013-06-19 15:26:41+01:00     0
2013-06-19 15:26:45+01:00     6
2013-06-19 15:26:51+01:00     0
2013-06-19 15:26:58+01:00     0
2013-06-19 15:27:04+01:00    26
Name: (power, active), dtype: float32

or, to get reactive power:

In [17]:
series = next(fridge.power_series(ac_type='reactive'))
series.head()

MeasurementError: AC type 'reactive' not available. Available columns = [('power', 'active')]. 

### Specify physical_quantity or AC type

In [7]:
df = next(fridge.load(physical_quantity='power', ac_type='reactive'))
df.head()

physical_quantity,power
type,reactive
2013-06-07 05:30:00+05:30,2.483
2013-06-07 05:30:01+05:30,2.547
2013-06-07 05:30:02+05:30,2.480
2013-06-07 05:30:03+05:30,2.444
2013-06-07 05:30:04+05:30,2.510


To load voltage data:

In [8]:
df = next(fridge.load(physical_quantity='voltage'))
df.head()

physical_quantity,voltage
type,
2013-06-07 05:30:00+05:30,235.070007
2013-06-07 05:30:01+05:30,235.020004
2013-06-07 05:30:02+05:30,234.979996
2013-06-07 05:30:03+05:30,235.000000
2013-06-07 05:30:04+05:30,234.949997


In [9]:
df = next(fridge.load(physical_quantity = 'power'))
df.head()

physical_quantity            power                
type                      apparent active reactive
2013-06-07 05:30:00+05:30    2.486  0.111    2.483
2013-06-07 05:30:01+05:30    2.555  0.200    2.547
2013-06-07 05:30:02+05:30    2.485  0.152    2.480
2013-06-07 05:30:03+05:30    2.449  0.159    2.444
2013-06-07 05:30:04+05:30    2.519  0.215    2.510

### Loading by specifying AC type

In [18]:
df = next(fridge.load(ac_type='active'))
df.head()

physical_quantity,power
type,active
2013-06-19 15:26:41+01:00,0
2013-06-19 15:26:45+01:00,6
2013-06-19 15:26:51+01:00,0
2013-06-19 15:26:58+01:00,0
2013-06-19 15:27:04+01:00,26


### Loading by resampling to a specified period

In [19]:
# resample to minutely (i.e. with a sample period of 60 secs)
df = next(fridge.load(ac_type='active', sample_period=60))
df.head()

physical_quantity,power
type,active
2013-06-19 15:26:00+01:00,1.500000
2013-06-19 15:27:00+01:00,25.000000
2013-06-19 15:28:00+01:00,24.222221
2013-06-19 15:29:00+01:00,24.111111
2013-06-19 15:30:00+01:00,24.000000
